# Linear Programming 3: Integer Linear Programming

## Cutting paper roles, again

We will take the last example from Chapter 2, but now instead use integer variables. This is very simple to achieve:

- We will use a different solver in OR-tools: The standard MIP solver in OR-Tools is SCIP
- Instead of NumVar we write IntVar

In [4]:
# based on: https://developers.google.com/optimization/mip/mip_var_array

from ortools.linear_solver import pywraplp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["constraint_coeffs"] = [
        [2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 2, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 2, 1, 0, 3, 2, 1, 0],
        [0, 1, 1, 3, 2, 0, 2, 4, 0, 2, 4, 7],
    ]
    data["bounds"] = [97, 610, 395, 211]
    data["obj_coeffs"] = [1] * 12
    data["num_vars"] = 12
    data["num_constraints"] = 4
    return data

data = create_data_model()

solver = pywraplp.Solver.CreateSolver("SCIP")

infinity = solver.infinity()
x = {}
for j in range(data["num_vars"]):
    x[j] = solver.IntVar(0, infinity, "x[%i]" % j)
print("Number of variables =", solver.NumVariables())

for i in range(data["num_constraints"]):
    constraint = solver.RowConstraint(data["bounds"][i], infinity, "")
    for j in range(data["num_vars"]):
        constraint.SetCoefficient(x[j], data["constraint_coeffs"][i][j])
print("Number of constraints =", solver.NumConstraints())
# In Python, you can also set the constraints as follows.
# for i in range(data['num_constraints']):
#  constraint_expr = \
# [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
#  solver.Add(sum(constraint_expr) <= data['bounds'][i])

objective = solver.Objective()
for j in range(data["num_vars"]):
    objective.SetCoefficient(x[j], data["obj_coeffs"][j])
objective.SetMinimization()
# In Python, you can also set the objective as follows.
# obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
# solver.Maximize(solver.Sum(obj_expr))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Objective value =", solver.Objective().Value())
    for j in range(data["num_vars"]):
        print(x[j].name(), " = ", x[j].solution_value())
    print()
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())
else:
    print("The problem does not have an optimal solution.")


Number of variables = 12
Number of constraints = 4
Objective value = 453.0
x[0]  =  0.0
x[1]  =  97.0
x[2]  =  0.0
x[3]  =  0.0
x[4]  =  158.0
x[5]  =  198.0
x[6]  =  0.0
x[7]  =  0.0
x[8]  =  0.0
x[9]  =  0.0
x[10]  =  0.0
x[11]  =  0.0

Problem solved in 15.000000 milliseconds
Problem solved in 3 iterations
Problem solved in 1 branch-and-bound nodes
